# Parameter Efficient Fine-tuning of the Gemma model with QLORA

# **Step 1: Install All the Required Packages**

In [1]:
!pip install peft datasets transformers trl accelerate bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.8 MB/s eta 0:00:00


# **Step 2: Import All the Required Libraries**

In [2]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModelForCausalLM
from trl import SFTTrainer
from datasets import Dataset
from huggingface_hub import notebook_login


# **Step 3: HuggingFace loging with token**

In [66]:
notebook_login()

# **Step 4: Data preperation**

In [3]:
f = open(r"/content/english_python_data.txt", "r", encoding="utf8")
file_lines = f.readlines()

In [4]:
file_lines[:7]

['# write a python program to add two numbers \n',
 'num1 = 1.5\n',
 'num2 = 6.3\n',
 'sum = num1 + num2\n',
 "print(f'Sum: {sum}')\n",
 '\n',
 '\n']

In [5]:
dps = []
dp = None
for line in file_lines:
    if line[0] == "#":
        if dp:
            dp['solution'] = ''.join(dp['solution'])
            dps.append(dp)
        dp = {"question": None, "solution": []}
        dp['question'] = line[1:]
    else:
        dp["solution"].append(line)

In [6]:
dps

[{'question': ' write a python program to add two numbers \n',
  'solution': "num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n\n\n"},
 {'question': ' write a python function to add two user provided numbers and return the sum\n',
  'solution': 'def add_two_numbers(num1, num2):\n    sum = num1 + num2\n    return sum\n\n\n'},
 {'question': ' write a program to find and print the largest among three numbers\n',
  'solution': "\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >= num2) and (num1 >= num3):\n   largest = num1\nelif (num2 >= num1) and (num2 >= num3):\n   largest = num2\nelse:\n   largest = num3\nprint(f'largest:{largest}')\n\n\n"},
 {'question': ' write a program to find and print the smallest among three numbers\n',
  'solution': "num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= num2) and (num1 <= num3):\n   smallest = num1\nelif (num2 <= num1) and (num2 <= num3):\n   smallest = num2\nelse:\n   smallest = num3\nprint(f'smallest:{smallest}')\n\n\n"},
 {'question': ' Wri

In [7]:
data_python_code = []
for dp in dps:
    data_python_code.append(f"Instruction:\n{dp['question'][1:]}\n\nResponse:\n{dp['solution']}")

In [19]:
len(data_python_code)

4957

In [20]:
data_python_code

["Instruction:\nwrite a python program to add two numbers \n\n\nResponse:\nnum1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n\n\n",
 'Instruction:\nwrite a python function to add two user provided numbers and return the sum\n\n\nResponse:\ndef add_two_numbers(num1, num2):\n    sum = num1 + num2\n    return sum\n\n\n',
 "Instruction:\nwrite a program to find and print the largest among three numbers\n\n\nResponse:\n\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >= num2) and (num1 >= num3):\n   largest = num1\nelif (num2 >= num1) and (num2 >= num3):\n   largest = num2\nelse:\n   largest = num3\nprint(f'largest:{largest}')\n\n\n",
 "Instruction:\nwrite a program to find and print the smallest among three numbers\n\n\nResponse:\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= num2) and (num1 <= num3):\n   smallest = num1\nelif (num2 <= num1) and (num2 <= num3):\n   smallest = num2\nelse:\n   smallest = num3\nprint(f'smallest:{smallest}')\n\n\n",
 'Instruction:\nWrite a python func

In [22]:
from datasets import Dataset

In [21]:
dataset ={'text': data_python_code}


In [23]:
dataset["text"]

["Instruction:\nwrite a python program to add two numbers \n\n\nResponse:\nnum1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n\n\n",
 'Instruction:\nwrite a python function to add two user provided numbers and return the sum\n\n\nResponse:\ndef add_two_numbers(num1, num2):\n    sum = num1 + num2\n    return sum\n\n\n',
 "Instruction:\nwrite a program to find and print the largest among three numbers\n\n\nResponse:\n\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >= num2) and (num1 >= num3):\n   largest = num1\nelif (num2 >= num1) and (num2 >= num3):\n   largest = num2\nelse:\n   largest = num3\nprint(f'largest:{largest}')\n\n\n",
 "Instruction:\nwrite a program to find and print the smallest among three numbers\n\n\nResponse:\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= num2) and (num1 <= num3):\n   smallest = num1\nelif (num2 <= num1) and (num2 <= num3):\n   smallest = num2\nelse:\n   smallest = num3\nprint(f'smallest:{smallest}')\n\n\n",
 'Instruction:\nWrite a python func

In [24]:
len(dataset["text"])

4957

In [25]:
dataset = Dataset.from_dict(dataset)


In [45]:
dataset

Dataset({
    features: ['text'],
    num_rows: 4957
})

In [58]:
dataset_train = dataset.select(range(1000))

In [59]:
dataset_val = dataset.select(range(1000, 1200))

In [60]:
len(dataset_train), len(dataset_val)

(1000, 200)

In [61]:
dataset_train

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [62]:
dataset_train["text"][0]

"Instruction:\nwrite a python program to add two numbers \n\n\nResponse:\nnum1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n\n\n"

In [63]:
dataset_val

Dataset({
    features: ['text'],
    num_rows: 200
})

In [64]:
dataset_val["text"][0]

'Instruction:\n write a program to get numbers = 1,3,11,42,12,4001\n\n\nResponse:\nfrom collections import Iterable\nhighestnumber = -999\nfor i in numbers:\n  if i > highestnumber:\n    highestnumber = i\nprint(numbers.index(highestnumber))\n\n'

# **Step 4: Load Model - BitsAndBytesConfig int-4 config**

In [67]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
tokenizer.padding_side = 'right'
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

# **Step 5: Lora congif**

In [68]:
peft_config = LoraConfig(
    lora_alpha = 16,
    lora_dropout=0.1,
    r=16,
    task_type='CAUSAL_LM'
)

# **Step 6: Initialize the Supervised Fine-tuning Trainer**

In [69]:
training_arguments = TrainingArguments(
    output_dir = "/PATH_TO_TRAINING_OUTPUT",
    evaluation_strategy="steps",
    logging_strategy="steps",
    lr_scheduler_type="constant",
    logging_steps=20,
    eval_steps=20,
    save_steps=20,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    eval_accumulation_steps=16,
    num_train_epochs=5,
    fp16=True,
    group_by_length = True,
    optim="paged_adamw_32bit",
    max_steps = 100
)
trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    dataset_text_field='text',
    peft_config=peft_config,
    neftune_noise_alpha=5,
    max_seq_length=500,
    args = training_arguments
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:236: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
20,3.916500,3.331199
40,3.456500,2.683542
60,2.712800,2.104791
80,2.112900,1.780471
100,1.851800,1.650420


TrainOutput(global_step=100, training_loss=2.8101186752319336, metrics={'train_runtime': 700.7346, 'train_samples_per_second': 4.567, 'train_steps_per_second': 0.143, 'total_flos': 3170575978831872.0, 'train_loss': 2.8101186752319336, 'epoch': 3.2})

In [70]:
trainer.model.save_pretrained("/SAVE_DIR")

In [71]:
finetuned_model = PeftModelForCausalLM.from_pretrained(model=model, model_id="/SAVE_DIR")

In [72]:
messages = [
    {
        'role':'user',
        'content':'Write a python function that returns the sum of n natural numbers?',
    }
]



input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_tensors="pt").to("cuda")

# print(input_ids)
outputs_finetuned = finetuned_model.generate(input_ids=input_ids, max_new_tokens=1024, do_sample=False)


print("finetuned: " + tokenizer.decode(outputs_finetuned[0]).split('<start_of_turn>model\n')[-1])


finetuned: ```python
def sum_natural_numbers(n):
    """
    Returns the sum of n natural numbers.

    Args:
        n: The number of natural numbers to sum.

    Returns:
        int: The sum of n natural numbers.
    """

    # Initialize the sum to 0.
    sum = 0

    # Iterate over the numbers from 1 to n.
    for i in range(1, n + 1):
        # Add the current number to the sum.
        sum += i

    # Return the sum.
    return sum


if __name__ == "__main__":
    # Get the number of natural numbers to sum from the user.
    n = int(input("Enter the number of natural numbers to sum: "))

    # Calculate and print the sum.
    sum = sum_natural_numbers(n)
    print(f"The sum of {n} natural numbers is {sum}")
```

**Example Usage:**

```
Enter the number of natural numbers to sum: 10
The sum of 10 natural numbers is 55
```

**Notes:**

* The function assumes that `n` is a positive integer.
* The time complexity of this function is O(n), where n is the number of natural numbers to 

In [75]:
messages = [
    {
        'role':'user',
        'content':'Write a Program to implement validation of a Password',
    }
]



input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_tensors="pt").to("cuda")

# print(input_ids)
outputs_finetuned = finetuned_model.generate(input_ids=input_ids, max_new_tokens=1024, do_sample=False)


print("finetuned: " + tokenizer.decode(outputs_finetuned[0]).split('<start_of_turn>model\n')[-1])

finetuned: ```python
import re


def validate_password(password):
    """
    Validates a password according to the password policy.

    Args:
        password (str): The password to validate.

    Returns:
        bool: True if the password is valid, False otherwise.
    """

    # Check if the password meets the minimum length requirement.
    if len(password) < 8:
        return False

    # Check if the password contains a mix of upper and lower case letters.
    if not re.search(r'[A-z]', password):
        return False

    # Check if the password contains a number.
    if not re.search(r'\d', password):
        return False

    # Check if the password contains a special character.
    if not re.search(r'!@#$%^&*_~', password):
        return False

    # Check if the password is the same as the original.
    if password == password.lower():
        return False

    return True


def main():
    """
    Gets the password from the user and validates it.
    """

    password = 